In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import pandas
import os
from google.colab import files

In [ ]:
def filter(contours, min_area):
  #filters contours based on area
  filtered_contours = []
  for contour in contours:
    if cv2.contourArea(contour) > min_area:
      filtered_contours.append(contour)
  return filtered_contours

def max_parent(contours, contours_hierarchy):
  #gets the index of the maximum parent contour
  parent_contours = []
  areas = []
  for contour in contours:
    areas.append(cv2.contourArea(contour))
  for i in range(len(contours)):
    if contours_hierarchy[0][i][3] == -1:
      parent_contours.append(cv2.contourArea(contours[i]))
  if parent_contours:
    max_contour = max(parent_contours)
    return areas.index(max_contour)

def get_family(contours, contours_hierarchy, parent_index):
  #gets the child contours of the parent contour
  child_contours = []
  for i in range(len(contours)):
    if contours_hierarchy[0][i][3] == parent_index:
      child_contours.append(contours[i])
  if child_contours:
    return child_contours


def get_nucleus(blue_contours, parent_contour):
  #determines which blue contours are nucleus from the parent cell
  nucleus = []
  for contour in blue_contours:
    for point in contour:
      x, y = point[0]
      if cv2.pointPolygonTest(parent_contour, (float(x), float(y)), False) != -1:
        nucleus.append(contour)
        break
  if nucleus:
    return nucleus

In [ ]:
file_list = []
for file_path in os.listdir('/content'):
    # check if current file_path is a file
    if os.path.isfile(os.path.join('/content', file_path)):
        # add filename to list
        file_list.append(file_path)

folder_path = '/content/measurements'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

#Usuário submete valores mínimos para operações nas imagens
min_cell_area = int(input("Digite a área mínima para uma célula: "))
min_nucleus_area = int(input("Digite a área mínima para um núcleo: "))

for filename in file_list:
  img = cv2.imread(filename)
  img=cv2.copyMakeBorder(img, 5, 5, 5, 5, cv2.BORDER_CONSTANT)
  cv2_imshow(img)
  red = np.copy(img[:,:,2])

  #filters and analyses of red channel, for cropping
  first_otsu, first_thresh = cv2.threshold(red, 0, 255, cv2.THRESH_BINARY + \
                                          cv2.THRESH_OTSU)
  first_contours, first_hierarchy = cv2.findContours(first_thresh, cv2.RETR_EXTERNAL,\
                                                    cv2.CHAIN_APPROX_NONE)
  first_contours = filter(first_contours, min_cell_area)

  #starts cropping the image
  dataframe_data = []
  thresh_value = first_otsu/2
  for contour in first_contours:
    x, y, w, h = cv2.boundingRect(contour)
    scrap = img[y:y+h, x:x+w]
    scrap = cv2.copyMakeBorder(scrap, 5, 5, 5, 5, cv2.BORDER_CONSTANT)
    red_scrap = scrap[:, :, 2]
    blue_scrap = scrap[:, :, 0]

    #dataframe stuff
    new_row_data = {'cell_area': '', 'nuclear_area': ''}

    #filters red channel crop
    red_otsu, red_thresh = cv2.threshold(red_scrap, thresh_value, 255, cv2.THRESH_BINARY)
    red_thresh = cv2.dilate(red_thresh, (3,3), iterations=1)
    red_contours, red_hierarchy = cv2.findContours(red_thresh, cv2.RETR_TREE,\
                                                  cv2.CHAIN_APPROX_NONE)
    parent = max_parent(red_contours, red_hierarchy)
    children = get_family(red_contours, red_hierarchy, parent)
    parent_area = cv2.contourArea(red_contours[parent])
    cv2.drawContours(scrap, red_contours, parent, (0, 0, 255), 1)
    try:
      for child in children:
        parent_area = parent_area - cv2.contourArea(child)
    except TypeError:
      pass

    #filters blue channel crop
    blue_otsu, blue_thresh = cv2.threshold(blue_scrap, first_otsu, 255, cv2.THRESH_BINARY)
    blue_thresh = cv2.dilate(blue_thresh, (3,3), iterations=1)
    blue_contours, blue_hierarchy = cv2.findContours(blue_thresh, cv2.RETR_TREE,\
                                                    cv2.CHAIN_APPROX_NONE)
    blue_contours = filter(blue_contours, min_nucleus_area)
    nucleus = get_nucleus(blue_contours, red_contours[parent])
    nucleus_areas = []
    if nucleus:
      for nuclei in nucleus:
        nucleus_areas.append(cv2.contourArea(nuclei))
    cv2.drawContours(scrap, nucleus, -1, (0, 255, 0), 1)
    cv2_imshow(scrap)

    #dataframe stuff
    new_row = {'Cell Area': parent_area, 'Nucleus Area': sum(nucleus_areas),\
              'Clear Nuclei Count': len(nucleus_areas), 'Nuclear Area Fraction':\
              sum(nucleus_areas) / parent_area * 100}
    dataframe_data.append(new_row)
    print(new_row)
  df = pandas.DataFrame(dataframe_data, columns=['Cell Area', 'Nucleus Area',\
                                                'Clear Nuclei Count',\
                                                'Nuclear Area Fraction'])

  df = df[df > 0].dropna()
  df.style
  df.to_csv(f'{folder_path}/results_{filename}.csv', index=False)
  files.download(f'{folder_path}/results_{filename}.csv')